In [3]:
# !pip3 install numpy
# !pip3 install pandas
# !pip3 install csv
!pip3 install openpyxl

In [4]:
import pandas as pd
import numpy as np
import csv
import re

north = pd.read_excel(r"datasets\พจนานุกรมภาษาถิ่นเหนือ.xlsx", header=None)
northeast = pd.read_excel(r"datasets\พจนานุกรมภาษาถิ่นอีสาน.xlsx", header=None)
south = pd.read_excel(r"datasets\พจนานุกรมภาษาถิ่นใต้.xlsx", header=None)

In [8]:
south_dict = south.to_dict(orient='records')

def shift_values_left(d):
    values = [v for v in d.values() if pd.notna(v)]
    return {i: values[i] if i < len(values) else np.nan for i in d.keys()}

south = pd.DataFrame([shift_values_left(d) for d in south_dict])

first_col = south.iloc[:, 1]

0       น. กก, ชื่อไม้ล้มลุกชนิดหนึ่งเกิดในที่ชุ่มน้ำ ...
1       น. กกลังกาชนิดหนึ่ง แต่ต้นเล็กกว่า ใช้เป็นสมุนไพร
2                          น. กกชนิดต้นกลม ใช้เป็นสมุนไพร
3                                                     NaN
4       ว. มาก, ยิ่ง เช่น แก่กกกก - แก่มาก, แห้งกกกก -...
                              ...                        
1717                            น. กรรไกรแบบจีน ใช้ตัดผ้า
1718             น. กรรไกรที่ทำหัวเป็นรูปหงส์ ; สำ. โบราณ
1719                                             ก็เรียก.
1720                                                  NaN
1721                                 น. รกมะพร้าว (ช.พ.).
Name: 1, Length: 1722, dtype: object

In [6]:
def split_definition(row):
    word = str(row[0]) if pd.notna(row[0]) else ''
    definition = str(row[1]) if pd.notna(row[1]) else ''
    if not isinstance(definition, str):
        return [[word, '', str(definition)]]
    parts = re.split(r'(?:(?<=\s)|^)(?=(?:น\.|ก\.|ว\.|สัน\.|สำ\.|ลัก\.)|\d+\s+(?:น\.|ก\.|ว\.|สัน\.|สำ\.|ลัก\.))', definition)
    result = []
    current_pos = ''
    for part in parts:
        pos_match = re.match(r'^(?:\d+\s+)?(น\.|ก\.|ว\.|สัน\.|สำ\.|ลัก\.)', part)
        if pos_match:
            current_pos = pos_match.group(1)
            def_part = part[pos_match.end():].strip()
        else:
            def_part = part.strip()
        sub_parts = re.split(r'(?<=\.)\s*(?:\d+\.|\d+\)\.)\s*', def_part)
        for sub_part in sub_parts:
            if sub_part.strip():
                result.append([word, current_pos.strip(), sub_part.strip()])
    return result if result else [[word, '', definition]]

In [ ]:
# ver1

# def split_definition(row):
#     word = row[0]
#     definition = row[1] if pd.notna(row[1]) else ''
#     parts = re.split(r',\s*(?=(?:น\.|ก\.|ว\.|สัน\.|สำ\.|ลัก\.))', definition) 
#     result = []
#     for part in parts:
#         match = re.match(r'(น\.|ก\.|ว\.|สัน\.|สำ\.|ลัก\.)\s*(.*)', part)
#         if match:
#             pos, def_part = match.groups()
#             sub_parts = re.split(r'(?<=\.)\s*\d+\)\.\s*', def_part)
#             for sub_part in sub_parts:
#                 if sub_part.strip():
#                     result.append([word, pos.strip(), sub_part.strip()])
#         else:
#             result.append([word, '', part.strip()])
#     return result

# ver 2

# def split_definition(row):
#     word = str(row[0]) if pd.notna(row[0]) else ''
#     definition = str(row[1]) if pd.notna(row[1]) else ''
#     if not isinstance(definition, str):
#         return [[word, '', str(definition)]]
#     parts = re.split(r'(?:(?<=\s)|^)(?=(?:น\.|ก\.|ว\.|สัน\.|สำ\.|ลัก\.)|\d+\s+(?:น\.|ก\.|ว\.|สัน\.|สำ\.|ลัก\.))', definition)
#     result = []
#     current_pos = ''
#     for part in parts:
#         pos_match = re.match(r'^(?:\d+\s+)?(น\.|ก\.|ว\.|สัน\.|สำ\.|ลัก\.)', part)
#         if pos_match:
#             current_pos = pos_match.group(1)
#             def_part = part[pos_match.end():].strip()
#         else:
#             def_part = part.strip()
#         sub_parts = re.split(r'(?<=\.)\s*(?:\d+\.|\d+\)\.)\s*', def_part)
#         for sub_part in sub_parts:
#             if sub_part.strip():
#                 result.append([word, current_pos.strip(), sub_part.strip()])
#     return result if result else [[word, '', definition]]

# ver 3

# def split_definition(row):
#     word = str(row[0]) if pd.notna(row[0]) else ''
#     definition = str(row[1]) if pd.notna(row[1]) else ''
    
#     if not isinstance(definition, str):
#         return [word, '', '', str(definition)]
    
#     def remove_thai_numbers(s):
#         return re.sub(r'^[๑๒๓๔๕๖๗๘๙๐]+\.\s*', '', s)
    
#     pos_splits = re.split(r'(?:(?<=\s)|^)(?=(น\.|ก\.|ว\.|สัน\.|สำ\.|ลัก\.))', definition)
    
#     alternative_forms = []
#     pos_definitions = []
    
#     for i, part in enumerate(pos_splits):
#         if part in ['น.', 'ก.', 'ว.', 'สัน.', 'สำ.', 'ลัก.']:
#             current_pos = part
#         else:
#             number_splits = re.split(r'(?:(?<=\s)|^)(?=[๑๒๓๔๕๖๗๘๙๐]+\.)', part)
            
#             for sub_part in number_splits:
#                 clean_part = remove_thai_numbers(sub_part.strip())
#                 if clean_part:
#                     if i == 0:
#                         alternative_forms.append(clean_part)
#                     else:
#                         pos_definitions.append((current_pos.strip(), clean_part))
    
#     alt_forms_str = ', '.join(alternative_forms) if alternative_forms else ''
    
#     if pos_definitions:
#         pos_str = pos_definitions[0][0]
#         def_str = ', '.join([d[1] for d in pos_definitions])
#         return [word, alt_forms_str, pos_str, def_str]
#     else:
#         return [word, alt_forms_str, '', '']